In [12]:
import numpy as np
import faiss
import matplotlib.pyplot as plt
exec(open("../database.py").read())
exec(open("../helperFunctions.py").read())
import pandas as pd
from keras.datasets import mnist
import math
from time import perf_counter

***Prepare dataset***

In [13]:
def createTrueDatasets (name, metric, queries) :
    datasetTrainImages, datasetTestImages, _ = get_ann_benchmark_data(name)
    index = faiss.IndexFlatL2(datasetTrainImages.shape[1])   # take basic index
    print('index.is_trained : ', index.is_trained)

    index.add(datasetTrainImages) # add vectors to the index
    print('index.ntotal : ', index.ntotal)

    D = []
    I = []
    def createTrueLabels(par, D, I, datasetTestImages):
        k = 100
        totalTime = 0
        for i in range(par) : 
            xq = datasetTestImages[i:i+1] # Use the first image as the query vector
            time_start = perf_counter()
            x, y = index.search(xq, k)
            time_end = perf_counter()
            totalTime += (time_end - time_start)
            D.append(np.sqrt(x[0]))
            I.append(y[0])
        print(totalTime)
        indexes = np.array(I)
        distances = np.array(D)
        naming = name + '-' + metric + '-true-labels.xlsx'
        exportDB(indexes, distances, naming, 1)
        
    createTrueLabels(queries, D, I, datasetTestImages)


In [14]:
datasets_list = ['deepImage-96', 'fashionMnist-784', 'gist-960', 'glove-25', 'glove-50', 'glove-100', 'glove-200', 'mnist-784', 'nytimes-256', 'sift-128', 'lastfm-64']
specialDatasets = ['kosarak', 'movielens10m']
examle = ['mnist-784']
metrics = ['euclidean']
queries = 1000
for j in metrics:
    for i in datasets_list:
        createTrueDatasets(i, j, queries)

/home/pb_ra/Angsar_Aidarbek_thesis/annSearchComparisons/datasets/deepImage-96.hdf5


OSError: Unable to synchronously open file (truncated file: eof = 64733184, sblock->base_addr = 0, stored_eof = 3848008288)